In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Chf_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + 'Oanda_Aud_Usd_M30_2021-2022.csv')
df_long.Date = pd.to_datetime(df_long.Date)
df_long.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-01-03 22:00:00   1.20492   1.20569  1.20492    1.20549   1.20642   
1 2021-01-03 22:05:00   1.20533   1.20567  1.20504    1.20517   1.20683   
2 2021-01-03 22:10:00   1.20523   1.20570  1.20523    1.20569   1.20673   
3 2021-01-03 22:15:00   1.20561   1.20569  1.20529    1.20541   1.20711   
4 2021-01-03 22:20:00   1.20535   1.20649  1.20535    1.20603   1.20685   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.20719  1.20642    1.20699   1.20567   1.20644  1.20567    1.20624   
1   1.20717  1.20654    1.20667   1.20608   1.20642  1.20579    1.20592   
2   1.20720  1.20673    1.20719   1.20598   1.20645  1.20598    1.20644   
3   1.20719  1.20679    1.20691   1.20636   1.20644  1.20604    1.20616   
4   1.20799  1.20685    1.20753   1.20610   1.20724  1.20610    1.20678   

   Volume  
0      30  
1      35  
2      10  
3       9  
4      27  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

In [6]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()
df['ema12'] = pd.Series.ewm(df['Mid_Close'], span=12).mean()
df['ema4'] = pd.Series.ewm(df['Mid_Close'], span=4).mean()

df['atr'] = atr(df)
df['rsi'] = rsi(df)
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=9).mean() - pd.Series.ewm(df['Mid_Close'], span=30).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=3).mean()
df['n_macd'], df['n_macdsignal'] = n_macd(df['macd'], df['macdsignal'])
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['sar'] = psar(df)

df['chop14'] = chop(df)
df['chop36'] = chop(df, lookback=36)

df['vo'] = vo(df['Volume'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)


df_long['ema200'] = pd.Series.ewm(df_long['Mid_Close'], span=200).mean()
df_long['ema100'] = pd.Series.ewm(df_long['Mid_Close'], span=100).mean()
df_long['ema50'] = pd.Series.ewm(df_long['Mid_Close'], span=50).mean()
df_long['ema25'] = pd.Series.ewm(df_long['Mid_Close'], span=25).mean()

df_long['atr'] = atr(df_long)
df_long['rsi'] = rsi(df_long)
df_long['rsi_sma'] = df_long['rsi'].rolling(50).mean()
df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['macd'] = pd.Series.ewm(df_long['Mid_Close'], span=12).mean() - pd.Series.ewm(df_long['Mid_Close'], span=26).mean()
df_long['macdsignal'] = pd.Series.ewm(df_long['macd'], span=9).mean()
df_long['n_macd'], df_long['n_macdsignal'] = n_macd(df_long['macd'], df_long['macdsignal'])
df_long['slowk'], df_long['slowd'] = stoch(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])

df_long['sar'] = psar(df_long)

df_long['chop14'] = chop(df_long)
df_long['chop36'] = chop(df_long, lookback=36)

df_long['vo'] = vo(df_long['Volume'])

df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-01-04 03:15:00   1.21006   1.21024  1.20996    1.21020   1.21030   
1     2021-01-04 03:20:00   1.21021   1.21023  1.20999    1.21012   1.21043   
2     2021-01-04 03:25:00   1.21011   1.21015  1.20992    1.20992   1.21033   
3     2021-01-04 03:30:00   1.20992   1.21074  1.20988    1.21071   1.21013   
4     2021-01-04 03:35:00   1.21071   1.21073  1.21027    1.21028   1.21096   
...                   ...       ...       ...      ...        ...       ...   
74489 2021-12-31 21:35:00   1.23282   1.23299  1.23273    1.23298   1.23312   
74490 2021-12-31 21:40:00   1.23298   1.23337  1.23298    1.23310   1.23329   
74491 2021-12-31 21:45:00   1.23310   1.23326  1.23303    1.23307   1.23349   
74492 2021-12-31 21:50:00   1.23308   1.23336  1.23281    1.23313   1.23349   
74493 2021-12-31 21:55:00   1.23316   1.23411  1.23236    1.23392   1.23366   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [11]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, use_chop, chop_cutoff, chop_num, use_rsi, use_long_ema, use_any_macd, tp_pips):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None

    for i in range(38, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_long = df_long.loc[df_long['Date'] < curr_date]
        if len(curr_long) == 0:
            continue
        ema200_2, ema100_2, ema50_2, ema25_2, atr2, sar2, rsi2, rsi_sma2, mid_open2, mid_close2, mid_low2, mid_high2, vo2, ema12_2, ema4_2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo', 'ema12', 'ema4']]
        ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, rsi_sma1, mid_open1, mid_close1, mid_low1, mid_high1, vo1, ema12_1, ema4_1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo', 'ema12', 'ema4']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        prev_bid_open, prev_ask_open = df.loc[df.index[i - 1], ['Bid_Open', 'Ask_Open']]
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open']]
        chop1 = df.loc[df.index[i - 1], f'chop{chop_num}']

        correct_time = 20 <= curr_date.hour <= 22 and 15 <= curr_date.minute <= 30 and curr_date.day != 4

        ema_200_long, chop_long = curr_long.loc[curr_long.index[-1], ['ema200', f'chop{chop_num}']]

        emas_buy_signal = ema_200_long < ema200_1 < mid_close1 if use_long_ema else ema200_1 < mid_close1
        emas_sell_signal = ema_200_long > ema200_1 > mid_close1 if use_long_ema else ema200_1 > mid_close1

        macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 if use_any_macd else macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
        macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 if use_any_macd else macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

        sar_buy_signal = sar1 < min([mid_open1, mid_close1])
        sar_sell_signal = sar1 > max([mid_open1, mid_close1])

        rsi_buy_signal = rsi1 > rsi_sma1 if use_rsi else True
        rsi_sell_signal = rsi1 < rsi_sma1 if use_rsi else True

        chop_signal = chop_long > chop_cutoff and chop1 < chop_cutoff if use_chop else True

        stoch_vals = list(df.loc[df.index[i - 12:], 'slowk_rsi'])
        stoch_vals_2 = list(df.loc[df.index[i - 12:], 'slowd_rsi'])

        stoch_buy_signal = True
        stoch_buy_changed = False
        stoch_sell_signal = True
        stoch_sell_changed = False

        # for zk in range(len(stoch_vals) - 1, -1, -1):
        #     if stoch_vals[zk] > 80 or stoch_vals_2[zk] > 80 and not stoch_buy_changed:
        #         stoch_buy_signal = False
        #         stoch_buy_changed = True

        #     if stoch_vals[zk] < 20 and stoch_vals_2[zk] < 20 and not stoch_buy_changed:
        #         stoch_buy_signal = True
        #         stoch_buy_changed = True

        #     if stoch_vals[zk] < 20 or stoch_vals_2[zk] < 20 and not stoch_sell_changed:
        #         stoch_sell_signal = False
        #         stoch_sell_changed = True

        #     if stoch_vals[zk] > 80 and stoch_vals_2[zk] > 80 and not stoch_sell_changed:
        #         stoch_sell_signal = True
        #         stoch_sell_changed = True

        bullish_candle = mid_close1 > mid_open1
        bearish_candle = mid_close1 < mid_open1

        mid_highs = list(df.loc[df.index[i - 38:i - 2], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - 38:i - 2], 'Mid_Low'])
        differences = []

        for i in range(len(mid_highs)):
            differences.append(abs(mid_highs[i] - mid_lows[i]))

        mean, std = np.array(differences).mean(), np.array(differences).std()

        bars_small_enough = abs(mid_high1 - mid_low1) <= mean + (std / 2) and abs(mid_high2 - mid_low2) <= mean + (std / 2)
        # bars_small_enough = True

        buy_signal = ema4_2 < ema12_2 and ema4_1 > ema12_1 and ema100_1 < mid_close1

        if trade is None:
            # if macd_buy_signal and emas_buy_signal and sar_buy_signal and rsi_buy_signal and chop_signal and stoch_buy_signal and bullish_candle and bars_small_enough:
            if buy_signal and chop_signal:
                open_price = float(curr_ask_open)
                pullback = sar1

                # stop_loss = round(pullback, 5)
                stop_loss = round(open_price - 0.0015, 5)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                      # if use_any_macd and max(macd_vals) >= 0:
                      #     stop_gain = round(open_price + (curr_pips_to_risk * 1.0), 5)

                      # else:
                      #     stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                      stop_gain = round(open_price + tp_pips, 5)

                      n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, 'AUD_USD')

                      trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                      'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                      'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                      n_buys += 1

                      pips_risked.append(curr_pips_to_risk)

            # elif macd_sell_signal and emas_sell_signal and sar_sell_signal and rsi_sell_signal and chop_signal and stoch_sell_signal and bearish_candle and bars_small_enough:
            elif False:
                open_price = float(curr_bid_open)
                pullback = sar1

                stop_loss = round(pullback, 5)
                # stop_loss = round(open_price + 0.0020, 5)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        if use_any_macd and min(macd_vals) <= 0:
                            stop_gain = round(open_price - (curr_pips_to_risk * 1.0), 5)

                        else:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                        # stop_gain = round(open_price - 0.0005, 5)

                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, 'AUD_USD')

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [12]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
# risk_reward_ratio_vals = [1.5, 2]
risk_reward_ratio_vals = [1.5]
# spread_cutoffs = [0.10, 0.15, 0.20]
spread_cutoffs = [0.20]
# use_chop_vals = [True, False]
use_chop_vals = [True]
# chop_cutoff_vals = [0.50, 0.55, 0.60]
chop_cutoff_vals = [0.50]
# chop_nums = [14, 36]
chop_nums = [14]
# use_rsi_vals = [True, False]
use_rsi_vals = [True]
# use_long_ema_vals = [True, False]
use_long_ema_vals = [False]
# use_any_macd_vals = [True, False]
use_any_macd_vals = [False]
tp_pips_vals = [0.0010]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_chop_vals) * len(chop_cutoff_vals) * len(chop_nums) * len(use_rsi_vals) * len(use_long_ema_vals) * len(use_any_macd_vals) * len(tp_pips_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for use_chop in use_chop_vals:
            for chop_cutoff in chop_cutoff_vals:
                for chop_num in chop_nums:
                    for use_rsi in use_rsi_vals:
                        for use_long_ema in use_long_ema_vals:
                            for use_any_macd in use_any_macd_vals:
                                for tp_pips in tp_pips_vals:
                                    all_combos.append((risk_reward_ratio, spread_val, use_chop, chop_cutoff, chop_num, use_rsi, use_long_ema, use_any_macd, tp_pips))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_chop_val = None
best_chop_cutoff_val = None
best_chop_num = None
best_use_rsi_val = None
best_use_long_ema_val = None
best_use_any_macd_val = None
best_tp_pips = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_chop, chop_cutoff, chop_num, use_rsi, use_long_ema, use_any_macd, tp_pips in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_chop, chop_cutoff, chop_num, use_rsi, use_long_ema, use_any_macd, tp_pips)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_chop': use_chop, 'chop_cutoff': chop_cutoff, 'chop_num': chop_num, 'use_rsi': use_rsi, 'use_long_ema': use_long_ema, 'use_any_macd': use_any_macd})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_use_chop_val = use_chop
        best_chop_cutoff_val = chop_cutoff
        best_chop_num = chop_num
        best_use_rsi_val = use_rsi
        best_use_long_ema_val = use_long_ema
        best_use_any_macd_val = use_any_macd
        best_tp_pips = tp_pips

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1

-1433.9856599999468
Num buys: 149
Num sells: 0
Num trades: 149
Num wins: 73
Num losses: 76
Win streak: 6
Loss streak: 7
Remaining runs: 0
Best reward so far: -1433.9856599999468



In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop val: ' + str(best_use_chop_val))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best chop val: ' + str(best_chop_num))
print('Best use rsi val: ' + str(best_use_rsi_val))
print('Best use long ema val: ' + str(best_use_long_ema_val))
print('Best tp val: ' + str(best_tp_pips))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 61.91590000001428
Best risk/reward ratio: 1.5
Best spread: 0.2
Best chop val: True
Best chop cutoff val: 0.5
Best chop val: 14
Best use rsi val: False
Best use long ema val: False
-----------------------
Top results:
{'reward': -354, 'ratio': 1.5, 'spread': 0.2, 'use_chop': True, 'chop_cutoff': 0.6, 'chop_num': 14, 'use_rsi': True, 'use_long_ema': True}
{'reward': -141, 'ratio': 1.5, 'spread': 0.2, 'use_chop': True, 'chop_cutoff': 0.55, 'chop_num': 14, 'use_rsi': False, 'use_long_ema': False}
{'reward': -345, 'ratio': 1.5, 'spread': 0.2, 'use_chop': True, 'chop_cutoff': 0.6, 'chop_num': 36, 'use_rsi': False, 'use_long_ema': True}
{'reward': -109, 'ratio': 1.5, 'spread': 0.2, 'use_chop': True, 'chop_cutoff': 0.55, 'chop_num': 14, 'use_rsi': False, 'use_long_ema': True}
{'reward': -382, 'ratio': 1.5, 'spread': 0.2, 'use_chop': True, 'chop_cutoff': 0.55, 'chop_num': 36, 'use_rsi': True, 'use_long_ema': True}
{'reward': -272, 'ratio': 1.